# PENNI framework workflow

In [4]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

from models import utils, vgg16
from models.op_count import profile
from decompose import params_resolver

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Baseline Model

In [12]:
model = vgg16.VGG16()
model.load_state_dict(torch.load("ckpt/Baseline/VGG16_93.49.pth"))
model.cuda()
utils.eval_cifar10(model)

==> Preparing data..
Files already downloaded and verified
Test Loss=0.3180, Test acc=0.9349


0.9349

## Decomposition and Retraining

In [13]:
resolver = params_resolver.param_resolver(model)
spar_model = resolver.PCA_decomposing(basis=5)
spar_model.cuda()

utils.train_cifar10(spar_model, lr=0.01, reg=1e-4, cross=True, cross_interval=2, spar_reg="l1", spar_param=1e-4)
utils.eval_cifar10(model)

Layer:features.0.weight ---- 	 64 x 3 Filters with shape (3, 3)
Layer:features.3.weight ---- 	 64 x 64 Filters with shape (3, 3)
Layer:features.7.weight ---- 	 128 x 64 Filters with shape (3, 3)
Layer:features.10.weight ---- 	 128 x 128 Filters with shape (3, 3)
Layer:features.14.weight ---- 	 256 x 128 Filters with shape (3, 3)
Layer:features.17.weight ---- 	 256 x 256 Filters with shape (3, 3)
Layer:features.20.weight ---- 	 256 x 256 Filters with shape (3, 3)
Layer:features.24.weight ---- 	 512 x 256 Filters with shape (3, 3)
Layer:features.27.weight ---- 	 512 x 512 Filters with shape (3, 3)
Layer:features.30.weight ---- 	 512 x 512 Filters with shape (3, 3)
Layer:features.34.weight ---- 	 512 x 512 Filters with shape (3, 3)
Layer:features.37.weight ---- 	 512 x 512 Filters with shape (3, 3)
Layer:features.40.weight ---- 	 512 x 512 Filters with shape (3, 3)
14 Conv Layers Loaded, have 4224 3D filters and 1634496 2D kernels in total.
Decomposing Layer: features.0.weight  with 5 Bas

KeyboardInterrupt: 

## Parameter and FLOPs Count

In [14]:
spar_model.cuda()
inputs = torch.randn(1,3,32,32).cuda()
flops = profile.profile(spar_model, [inputs,], verbose=False)
print(profile.clever_format(flops))

utils.compute_sparsity(spar_model)

183.40M
# of zero parameters:  0
# of coefficients:  8172480
# of total parameters:  8702473
Coefficient Sparsity:  0.0
Parameter Sparsity:  0.0


(0.0, 0.0)

## Prune and Finetune

In [16]:
utils.prune_by_std(spar_model)
utils.train_cifar10(spar_model, lr=0.01, reg=1e-4, epochs=30, finetune=True)
utils.eval_cifar10(spar_model)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Cannot find checkpoint folder!
Train From scratch!


KeyboardInterrupt: 

In [17]:
flops = profile.profile(spar_model, [inputs,], verbose=False)
print(profile.clever_format(flops))

utils.compute_sparsity(spar_model)

57.68M
# of zero parameters:  5848306
# of coefficients:  8172480
# of total parameters:  8702473
Coefficient Sparsity:  0.715609704765261
Parameter Sparsity:  0.672028054554148


(0.672028054554148, 0.715609704765261)

## Model Shrinking

In [24]:
spar_model = torch.load("ckpt/PrunedFinalModel/VGG16_pruned_9312.h5")
model_s, _ = utils.shrink(spar_model, iterative=True)

model_s.cuda()
utils.eval_cifar10(model_s)

Shrinking The Layer: features.0
0  |----->New input Channel 3  | New Output Channel 55  | New Basis 5
Shrinking The BN Layer: features.1
----->New number of features:  55
Shrinking The Layer: features.3
1  |----->New input Channel 55  | New Output Channel 64  | New Basis 5
Shrinking The BN Layer: features.4
----->New number of features:  64
Shrinking The Layer: features.7
2  |----->New input Channel 64  | New Output Channel 128  | New Basis 5
Shrinking The BN Layer: features.8
----->New number of features:  128
Shrinking The Layer: features.10
3  |----->New input Channel 128  | New Output Channel 128  | New Basis 5
Shrinking The BN Layer: features.11
----->New number of features:  128
Shrinking The Layer: features.14
4  |----->New input Channel 128  | New Output Channel 256  | New Basis 5
Shrinking The BN Layer: features.15
----->New number of features:  256
Shrinking The Layer: features.17
5  |----->New input Channel 256  | New Output Channel 256  | New Basis 5
Shrinking The BN Layer:

0.9313

In [25]:
flops = profile.profile(spar_model, [inputs,], verbose=False)
print(profile.clever_format(flops))

utils.compute_sparsity(spar_model)

22.56M
# of zero parameters:  8036663
# of coefficients:  8172480
# of total parameters:  8172480
Coefficient Sparsity:  0.983381176827597
Parameter Sparsity:  0.983381176827597


(0.983381176827597, 0.983381176827597)